<a href="https://colab.research.google.com/github/mehdii190/neural-network/blob/main/src/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


num_epochs = 30
batch_size = 100
learning_rate = 0.001




transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])


train_dataset = torchvision.datasets.CIFAR10(root = "../../data/",
                                             train = True,
                                             transform = transform,
                                             download = True)


test_dataset = torchvision.datasets.CIFAR10(root = "../../data/",
                                             train = False,
                                             transform = transforms.ToTensor())


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)



100%|██████████| 170498071/170498071 [00:02<00:00, 80406839.52it/s]


Extracting ../../data/cifar-10-python.tar.gz to ../../data/
